# TO DO
2. Now save the scrapped data into CSV and Sql lite Db ( Db name : MovieInfoDatabase Table name: MovieInfoTable) 
   Take user input and show the movie details asked for like director of the movie or the cast of the movie from movie_details
   varaible.
   - Satya  
   
4. Now after sorting the count and rating for the cast and directo we should now give out analysis of their career.
   - Satya
    
5. Find Cosine similarities between directors.
   - Manish 

# Tasks completed
1. Webscrapping of 500 movies and store them into dictionary
   - Manish
   
3. Now make a count of cast and director based on that give review ( we only did the cast part there is also director part)
     we store the cast & director data in varaibles : cast_details, director_details. 
   - Manish


# Task 1 : Webscraping
 1.import required libaries 
    * requests : To retrived the Html of the source page
    * bs4      : To converted the obtained html which in the form of text into soup object in which we can retrive specific                      data like headers, paragraphs, etc.
    * lxml     : Helps bs4 to understand the html.
    * csv      : To write or read into csv file.

In [45]:
import requests
import lxml
import bs4
import csv 
import time


2. Loading the featured page links into a link to hit them and creating headers to mimic actual browser.

In [56]:
urls =["https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc&page=0",
       "https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc&page=1",
       "https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc&page=2",
       "https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc&page=3",
       "https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc&page=4"]
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}



3. Initialise the varaible that are used to store the data

In [57]:
movie_short_href={} # used to store the href values 

movie_details={}    # Used to store the final movie details as 
                    #   { 'movie_name':{'Director':['director_name'], 
                    #                   'Principal Cast' : [name_1, name_2 ...name_m], 
                    #                    'Cast':['name_1, name_2... name_n'] }}  note m>n.
            
failed_items=[]     # Used to store the movie if of those movie where we have a failed attempt.




4. Below process is to achieve the link for each movies from the main page

    result : now we have collection of end points of the movies.

In [58]:

index_num=0
for url in urls:
    result = requests.get(url,headers=headers)
    soup = bs4.BeautifulSoup(result.text, "lxml")
    div_tag = soup.find_all('div', class_=["browse_list_wrapper one browse-list-large",
                                           "browse_list_wrapper two browse-list-large",
                                           "browse_list_wrapper three browse-list-large",
                                           "browse_list_wrapper four browse-list-large"])
    for item in div_tag:
        for element in item.find_all('td', class_='clamp-image-wrap'):
            index_num=index_num+1
            source=element.find_all('a' , href= True)[0]
            movie_short_href[index_num]=source['href'].replace('/movie/','')
    
    

5. This is method is to scrape individual movie details from its respective website, if there is any kind of failure while scraping we store the the movie id  in a list (failed_items).

In [59]:
def retrive_data_from_website(i):
    movie_link='https://www.metacritic.com/movie/'
    url = movie_link+movie_short_href[i]+'/details'
    result = requests.get(url,headers=headers)
    if result.status_code != 200:
        failed_items.append(i)
    else:
        
        final_details={}
        soup = bs4.BeautifulSoup(result.text,'lxml')
        tags = soup.find_all('table', class_='credits')
        wanted_columns=['Director', 'Principal Cast', 'Cast']
        movie_name = soup.find_all('div', class_='product_page_title oswald upper')[0].find_all('h1')[0].getText()
        meta_score = soup.find_all('a', class_ = 'metascore_anchor')[0].getText().strip()
        final_details['meta_score']=meta_score
        for table in tags:
            column = table.find_all('th', class_="person")[0].getText()
            if column in wanted_columns:
                details=[]
                for name in table.find_all('a', href=True):
                    details.append(name.getText().strip())
                final_details[column]=details
        if movie_name not in movie_short_href.values():
            movie_short_href[i] = movie_name
        movie_details[movie_short_href[i]] = final_details
    if i in [100,200,300,400,500]:
        time.sleep(2)
    if i in failed_items:
        failed_items.remove(i)
    

6. Now we hit every movie link from our collection and check for any failed attempts,
    if any there is any failed attempt, that implies that the href we obtained has different value which the website has
    internally rerouted, so just for the failed items instead of hitting the details  page we go step bye step and find the
    actual href value 
    
    example:
    movie href value = 'citizen-kane' in page - 1
    we would be hitting = '/movies/citizen-kane/details' which is wrong endpoint.
    actual endpoint = '/movies/citizen-kane-1941/details' 
    
   for movies like these which are less than 10 in count,
    we first hit the movie page which is '/movies/citizen-kane'  which would internal redirect at the server side into
   'movie/citizen-kane-1941' we now take the url and update the href value in movie_short_href variable and continue the 
    process
    

In [60]:

for i in movie_short_href.keys():
    retrive_data_from_website(i)
if len(failed_items)>0:
    for i in failed_items:
        url = 'https://www.metacritic.com/movie/'+movie_short_href[i]
        dummy = requests.get(url,headers={'User-Agent': 'Mozilla/5.0'})
        movie_short_href[i]=dummy.url[dummy.url.rindex('/')+1::]
        retrive_data_from_website(i)


# Task 2: Store into CSV and Sql lite Db


In [63]:
with open("dict.csv", "w", newline="") as csv_file:  
    writer = csv.writer(csv_file)
    for key, value in movie_details.items():
        writer.writerow([key, value])



8. Read from a csv file

In [ ]:
with open('dict.csv') as csv_file:
    reader = csv.reader(csv_file)
    mydict = dict(reader)


# Load the variable during dev phase 

Need to delete this when the project is completed. copy the cell from variable_pro.ipynb and run it under so we can load movie_details and movie_short_href

# Task
1. Analyze how many times has each actor/actress appeared in these top 500 movies, analyze how many times has each director appeared in these top 500 movies, what can that tell you about their career?
 to do 
  First extract all cast into a list :
  a) just the Principal cast?
  b) Principal cast +  cast?
  
  Second make a register for cast with their count attached  
  a) list of tuples  [(cast1,count),(cast2, count)...........]
  b) dictionary {{cast1:count}, {cast2: count}......... }
  c) just list  [cast1, cast2....][count1, count 2...] or [cast1, count1, cast2, count2..........]
  

# Task 3:List the names of the cast  and director that have appeared in different movies
1. Filter all the cast  and directors from each movie and append them to the list.
    cast_details={'name' :{count:'v',
                          rating:'v'}}
    director_details = {'name' :{count:'v',
                          rating:'v'}}
                          
     Count is the number of movies that they worked in.
     Rating would be the average of the rating of the all the movies that they worked in.
                               
                        


In [53]:
cast_details={}
director_details={}
def add_castnames_count(cast_names,score):
    for name in cast_names:
        cast_details[name]= cast_details.get(name,{'count':0, 'rating': 0})
        cast_details[name]['count'] = cast_details[name].get('count',1) + 1
        cast_details[name]['rating'] = cast_details[name]['rating']+int(score)

            
def add_directornames_count(director_name,score):
    for name in director_name:
        director_details[name]= director_details.get(name,{'count':0, 'rating': 0})
        director_details[name]['count'] = director_details[name].get('count',1) + 1
        director_details[name]['rating'] = director_details[name]['rating']+int(score)     

    


In [54]:
for movie_name,movies in movie_details.items():
    if 'Principal Cast' in movies.keys():
        add_castnames_count(movies['Principal Cast'], movies['meta-score'])
    if 'Cast' in movies.keys():
        add_castnames_count(movies['Cast'],movies['meta-score'])
    if 'Director' in movies.keys():
        add_directornames_count(movies['Director'],movies['meta-score'])
for name in cast_details.keys():
        cast_details[name]['rating'] = cast_details[name]['rating']/cast_details[name]['count']
for name in director_details.keys():
        director_details[name]['rating'] = director_details[name]['rating']/director_details[name]['count']
        


In [57]:

cast_details

{'Joseph Cotten': {'count': 5, 'rating': 94.2},
 'Orson Welles': {'count': 6, 'rating': 95.0},
 'Agnes Moorehead': {'count': 2, 'rating': 96.5},
 'Dorothy Comingore': {'count': 1, 'rating': 100.0},
 'Erskine Sanford': {'count': 2, 'rating': 96.5},
 'Everett Sloane': {'count': 1, 'rating': 100.0},
 'Fortunio Bonanova': {'count': 2, 'rating': 97.5},
 'George Coulouris': {'count': 1, 'rating': 100.0},
 'Georgia Backus': {'count': 3, 'rating': 94.0},
 'Gus Schilling': {'count': 1, 'rating': 100.0},
 'Harry Shannon': {'count': 2, 'rating': 94.5},
 'Paul Stewart': {'count': 2, 'rating': 93.5},
 'Philip Van Zandt': {'count': 1, 'rating': 100.0},
 'Ray Collins': {'count': 4, 'rating': 96.25},
 'Ruth Warrick': {'count': 1, 'rating': 100.0},
 'William Alland': {'count': 1, 'rating': 100.0},
 'Al Pacino': {'count': 5, 'rating': 91.4},
 'Marlon Brando': {'count': 7, 'rating': 92.57142857142857},
 'Abe Vigoda': {'count': 1, 'rating': 100.0},
 'Al Lettieri': {'count': 1, 'rating': 100.0},
 'Diane Ke

In [58]:
director_details

{'Orson Welles': {'count': 5, 'rating': 94.6},
 'Francis Ford Coppola': {'count': 5, 'rating': 93.6},
 'Alfred Hitchcock': {'count': 12, 'rating': 94.83333333333333},
 'Michael Curtiz': {'count': 4, 'rating': 91.5},
 'Richard Linklater': {'count': 3, 'rating': 94.66666666666667},
 'Krzysztof Kieslowski': {'count': 2, 'rating': 94.0},
 'Gene Kelly': {'count': 1, 'rating': 99.0},
 'Stanley Donen': {'count': 1, 'rating': 99.0},
 'Charles Chaplin': {'count': 2, 'rating': 94.5},
 'Barry Jenkins': {'count': 2, 'rating': 93.0},
 'D.W. Griffith': {'count': 1, 'rating': 99.0},
 'Ben Sharpsteen': {'count': 4, 'rating': 96.5},
 'Bill Roberts': {'count': 4, 'rating': 95.5},
 'Hamilton Luske': {'count': 2, 'rating': 97.5},
 'Jack Kinney': {'count': 2, 'rating': 97.5},
 'Norman Ferguson': {'count': 3, 'rating': 97.0},
 'T. Hee': {'count': 2, 'rating': 97.5},
 'Wilfred Jackson': {'count': 4, 'rating': 96.5},
 'John Huston': {'count': 5, 'rating': 92.8},
 'Guillermo del Toro': {'count': 2, 'rating': 9

# # Task4 : Finding cosine similarities between  Directors